<a href="https://colab.research.google.com/github/AfraAd/CSC413-Homeworks/blob/main/Fall25_Hw8_transformer_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 8

In this homework, you will train a multi-label text classifier on a subset of [AG News](https://huggingface.co/datasets/r-three/ag_news_subset) dataset using a pre-trained BERT model. The AG News dataset consists of news articles categorized into one of four topics (0 - World, 1 - Sports, 2 - Business, 3 - Sci/Tech).

**In part 1**, you will fine-tune a BERT-style model on the AG News dataset and evaluate its performance. You can find a tutorial for loading BERT and fine-tuning [here](https://huggingface.co/docs/transformers/training). For simplicity, I recommend using the [Hugging Face Transformers library](https://huggingface.co/docs/transformers/index).You're welcome to use a different framework if you prefer.

**In part 2**, instead of fine-tuning a BERT-style model directly, you will use the representations from the BERT-style model as input to a linear classifier. Does this approach perform better or worse?

For both part 1 and part 2, your goal is to achieve a test accuracy above the specified thresholds. You won’t have access to the test labels—just like in real-world applications!

**Tips about fine-tuning**

* Data preprocessing: raw text data should be tokenized before being fed to the model as batches during trainig.
* Hyperparameter choices: Experiment with settings such as learning rate, warmup ratio, optimizer, number of training steps, and batch size.
* Avoid overfitting: remember that your fine-tuned model will be evaluated on the test set!


**!! IMPORTANT NOTE !!**

You are free to explore and implement the training code however you want to maximize the model performance. However, please put the code you're running under `if __name__ == '__main__':` so that the particular training step is not run when we later evaluate your final script! Otherwise you may fail the Markus tests due to timeout.

```
if __name__ == '__main__':
    # your training code to fine-tune the model
    ...
```

# Part 1 (4 points)



In [1]:
if __name__ == '__main__':
    # !pip install datasets
    # !pip install evaluate
    # !pip install -U sentence-transformers

    from datasets import load_dataset, DatasetDict
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
    import numpy as np
    import pandas as pd
    from torch.utils.data import DataLoader
    import torch
    import evaluate

    from sentence_transformers import SentenceTransformer
    from sklearn.linear_model import LogisticRegression
    import joblib

    torch.cuda.empty_cache()
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    ################# Import additional packages you need #################
    #####################################################################################
    pass


In [2]:
################## HELPER CODE FOR SAVING RELEVANT FILES ##################
if __name__ == '__main__':
    def in_colab():
        try:
            import google.colab
            return True
        except ImportError:
            return False

    if in_colab():
        from google.colab import drive
        drive.mount('/content/drive')
        SAVE_PATH = '/content/drive/MyDrive/CSC413'
    else:
        SAVE_PATH = '.'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Part 1.a

Fine-tune [TinyBERT](https://huggingface.co/huawei-noah/TinyBERT_General_4L_312D) on AG News and evaluate the results. You can find a tutorial for loading BERT and fine-tuning [here](https://huggingface.co/docs/transformers/training). In that tutorial, you will need to change the dataset from `"yelp_review_full"` to the correct dataset path and the model from `"bert-base-uncased"` to `"huawei-noah/TinyBERT_General_4L_312D"`. You'll also need to modify the code since AG New is a four-class classification dataset (unlike the Yelp Reviews dataset, which is a five-class classification dataset).

**TODO**
* After fine-tuning the model, save model predictions on the test set to *part1_tiny_bert_model_test_prediction.csv*. The csv file should contain "index" columns, corresponding to the unique sample index, and "pred" column, the model prediction on that sample. Your model should achieve >= 80% on the test accuracy to receive a full mark.

```
index, pred
0,model_pred_value_0
1,model_pred_value_1
2,model_pred_value_2
...
```

In [3]:
######################## DO NOT MODIFY THE CODE ########################
if __name__ == '__main__':
    dataset = load_dataset('r-three/ag_news_subset')
    model = AutoModelForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", num_labels=4)
    tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
    print(dataset["train"][100])
#########################################################################
    # Tokenization function
    def tokenize_function(examples):
        # Combine title and description for richer context
        texts = [f"{title} {desc}" for title, desc in zip(examples['title'], examples['description'])]
        return tokenizer(texts, padding="max_length", truncation=True, max_length=128)

    # Tokenize datasets
    print("Tokenizing datasets...")
    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    # Data collator
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # Metric
    accuracy_metric = evaluate.load("accuracy")
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return accuracy_metric.compute(predictions=predictions, references=labels)

    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"{SAVE_PATH}/tinybert_results",
        eval_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=64,
        num_train_epochs=5,
        weight_decay=0.01,
        warmup_ratio=0.1,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        save_strategy="epoch",
        logging_steps=100,
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    # Train
    print("Training TinyBERT...")
    trainer.train()

    # Evaluate on validation set
    eval_results = trainer.evaluate()
    print(f"TinyBERT Validation Accuracy: {eval_results['eval_accuracy']:.4f}")

    # Predict on test set
    print("Generating TinyBERT predictions on test set...")
    test_predictions = trainer.predict(tokenized_datasets["test"])
    test_preds = np.argmax(test_predictions.predictions, axis=1)

    # Save predictions to CSV
    part1_tiny_bert_pred = pd.DataFrame({
        'index': range(len(test_preds)),
        'pred': test_preds
    })
    part1_tiny_bert_pred.to_csv(f"{SAVE_PATH}/part1_tiny_bert_model_test_prediction.csv", index=False)
    print(f"Saved TinyBERT predictions to {SAVE_PATH}/part1_tiny_bert_model_test_prediction.csv")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'label': 3, 'title': 'Microsoft Takes Giant Step against Spyware', 'description': 'Microsoft has beefed up its security portfolio by purchasing anti-spyware specialist Giant Company Software. A test version of a spyware protection, detection and removal tool based on Giant #39;s ', 'index': 100}
Tokenizing datasets...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/tmp/ipython-input-875063428.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Training TinyBERT...


wandb: Currently logged in as: afra-azad (afra-azad-university-of-toronto) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,1.296700,0.964475,0.703000
2,0.922600,0.708718,0.766000
3,0.732400,0.605773,0.836000
4,0.545400,0.538286,0.856000
5,0.508900,0.524473,0.856000


TinyBERT Validation Accuracy: 0.8560
Generating TinyBERT predictions on test set...


AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


**Your trianing code here...**

If your prediction is saved in pandas dataframe, you can do something like:
```
if __name__ == '__main__':
   part1_tiny_bert_pred.to_csv(f"{SAVE_PATH}/part1_tiny_bert_model_test_prediction.csv", index=False)
```

## Part 1.b

For this section, choose a different pre-trained BERT-style model from the [Hugging Face Model Hub](https://huggingface.co/models) and fine-tune it. There are tons of options - part of the homework is navigating the hub to find different models! I recommend picking a model that is smaller than BERT-Base (as TinyBERT is) just to make things computationally cheaper. Is the final validation accuracy higher or lower with this other model?

**TODO**
* As in part 1.a, save model predictions on the test set to *part1_hf_bert_model_test_prediction.csv*. The csv file should contain "index" columns, corresponding to the unique sample index, and "pred" column, the model prediction on that sample. Your model should achieve >=80% on the test accuracy to receive a full mark.

In [ ]:
if __name__ == '__main__':
    ############### YOUR CODE ###############
    # TODO: find a new HF BERT based model from HuggingFace and load it.
    HF_BERT_BASED_MODEL = "prajjwal1/bert-mini"

    # Load model and tokenizer
    print(f"Loading model: {HF_BERT_BASED_MODEL}")
    model_hf = AutoModelForSequenceClassification.from_pretrained(HF_BERT_BASED_MODEL, num_labels=4)
    tokenizer_hf = AutoTokenizer.from_pretrained(HF_BERT_BASED_MODEL)

    # Tokenization function for the new model
    def tokenize_function_hf(examples):
        texts = [f"{title} {desc}" for title, desc in zip(examples['title'], examples['description'])]
        return tokenizer_hf(texts, padding="max_length", truncation=True, max_length=128)

    # Metric function
    def compute_metrics_hf(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return accuracy_metric.compute(predictions=predictions, references=labels)

    # Tokenize datasets
    print("Tokenizing datasets with new tokenizer...")
    tokenized_datasets_hf = dataset.map(tokenize_function_hf, batched=True)

    # Data collator
    data_collator_hf = DataCollatorWithPadding(tokenizer=tokenizer_hf)

    # Training arguments
    training_args_hf = TrainingArguments(
        output_dir=f"{SAVE_PATH}/hf_bert_results",
        eval_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=64,
        num_train_epochs=5,
        weight_decay=0.01,
        warmup_ratio=0.1,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        save_strategy="epoch",
        logging_steps=100,
    )

    # Trainer
    trainer_hf = Trainer(
        model=model_hf,
        args=training_args_hf,
        train_dataset=tokenized_datasets_hf["train"],
        eval_dataset=tokenized_datasets_hf["validation"],
        tokenizer=tokenizer_hf,
        data_collator=data_collator_hf,
        compute_metrics=compute_metrics_hf,
    )

    # Train
    print(f"Training {HF_BERT_BASED_MODEL}...")
    trainer_hf.train()

    # Evaluate
    eval_results_hf = trainer_hf.evaluate()
    print(f"Validation Accuracy: {eval_results_hf['eval_accuracy']:.4f}")

    # Predict on test set
    print("Generating predictions on test set...")
    test_predictions_hf = trainer_hf.predict(tokenized_datasets_hf["test"])
    test_preds_hf = np.argmax(test_predictions_hf.predictions, axis=1)

    # Save predictions to CSV
    part1_hf_bert_pred = pd.DataFrame({
        'index': range(len(test_preds_hf)),
        'pred': test_preds_hf
    })
    part1_hf_bert_pred.to_csv(f"{SAVE_PATH}/part1_hf_bert_model_test_prediction.csv", index=False)
    print(f"Saved predictions to {SAVE_PATH}/part1_hf_bert_model_test_prediction.csv")
    #########################################

**Your training code here...**

Similarly, you can consider something like:

```
if __name__ == '__main__':
   part1_hf_bert_pred.to_csv(f"{SAVE_PATH}/part1_hf_bert_model_test_prediction.csv", index=False)
```

# Part 2 (2.5 points)

Instead of fine-tuning the full model on a target dataset, it's also possible to use the output representations from a BERT-style model as input to a linear classifier and *only* train the classifier (leaving the rest of the pre-trained parameters fixed). You can do this easily using the [`sentence-transformers`](https://www.sbert.net/) library. Using `sentence-tranformers` gives you back a fixed-length representation of a given text sequence. To achieve this, you need to
1. Pick a pre-trained sentence Transformer.
2. Load the AG News dataset and feed the text from each example into the model.
3. Train a linear classifier on the representations.
4. Evaluate performance on the validation set.

For the second step, you can learn more about how to use Hugging Face datasets [here](https://huggingface.co/docs/datasets/index). For the third and fourth step, it's possible to either do this directly in PyTorch, or collect the learned representations and use them as feature vectors to train a linear classifier in any other library (e.g. [scikit-learn](https://scikit-learn.org/stable/modules/linear_model.html)). For this homework, you will implement the second approach.

After you complete the above steps, is the accuracy on the validation set higher or lower using a fixed sentence Transformer?

**TODO**:
* Complete the `encode_data` function: the function embeds each text sample into an output representation using the provided sentence encoder. The function is called to map a text data sample to the model representation, as shown below:
```
dataset.map(lambda x: encode_data(sen_model, x), batched=True)
```
* Train a Logistic Regression classifier: use sklearn.linear_model.LogisticRegression to fit the model on the encoded text data.
* Save your trained model: After training, saved teh fitted logistic regression model as `sentence_encoder_classification.pkl`. Your model should achieve >=85% on the test accuracy to receive a full mark.

In [ ]:
def encode_data(model, x):
    """Takes the model and the dataset object
        Returns a dictionary consisting of "encoded_input" and "label" as keys.
        - "encoded_input" contains the tokenized text features produced by the sentence transformer.
        - "label" is the target class label for each example.
        encoded_input is the encoded text input, and label is the target label.
        NOTE: Please assume the dataset object is the original one loaded via
              load_dataset('r-three/') for reproducibility.
              Which means if you want to create additional features to create the encoded_input,
              do so within this function.
    """
    ####################### YOUR CODE ##########################
    # Combine title and description for richer text representation
    texts = [f"{title} {desc}" for title, desc in zip(x['title'], x['description'])]

    # Encode the texts using the sentence transformer
    embeddings = model.encode(texts, show_progress_bar=False)

    # Return dictionary with encoded input and labels
    d = {
        'encoded_input': embeddings,
        'label': x['label']
    }
    return d
    ############################################################

In [ ]:
########### PUT YOUR MODEL HERE ###########
SENTENCE_TRANSFORMER_MODEL = 'all-MiniLM-L6-v2'
###########################################

In [ ]:
########### DO NOT CHANGE THIS CODE ###########
if __name__ == "__main__":

    sen_model = SentenceTransformer(SENTENCE_TRANSFORMER_MODEL)
    # Prepare the dataset
    tokenized_dataset = dataset.map(lambda x: encode_data(sen_model, x), batched=True)
    print(tokenized_dataset['train'][100])
    X_train = np.stack([np.array(x['encoded_input']) for x in tokenized_dataset['train']])
    X_val = np.stack([np.array(x['encoded_input']) for x in tokenized_dataset['validation']])
    y_train = np.stack([np.array(x['label']) for x in tokenized_dataset['train']])
    y_val = np.stack([np.array(x['label']) for x in tokenized_dataset['validation']])

    print(X_train.shape)
    print(X_val.shape)
    print(y_train.shape)
    print(y_val.shape)


In [ ]:
########### COMPLETE THE FOLLOWING LOGISTIC REGRESSION CODE ###########
if __name__ == "__main__":
    # Train the logistic regression classifier on encoded training data
    print("Training Logistic Regression classifier...")
    classifier = LogisticRegression(
        max_iter=1000,
        C=1.0,
        solver='lbfgs',
        multi_class='multinomial',
        random_state=42,
        verbose=1
    )

    # Fit the classifier
    classifier.fit(X_train, y_train)

    # Evaluate on validation set
    val_accuracy = classifier.score(X_val, y_val)
    print(f"\nValidation Accuracy: {val_accuracy:.4f}")

    # Generate predictions on test set
    test_preds = classifier.predict(X_val)


In [ ]:
######################## TO SUBMIT ########################
if __name__ == "__main__":
    # Save the trained model
    model_path = f"{SAVE_PATH}/sentence_encoder_classification.pkl"

    joblib.dump(classifier, model_path)

    print(f"Saved model to {model_path}")
    print(f"Part 1.a predictions saved to {SAVE_PATH}/part1_tiny_bert_model_test_prediction.csv")
    print(f"Part 1.b predictions saved to {SAVE_PATH}/part1_hf_bert_model_test_prediction.csv")